## Data Exploration
Parse through 'yelp_academic_dataset_business.json' and check to see which location/cuisine combination has more restaurants.

## IMPORT PACKAGES

In [1]:
import pandas as pd
import pandasql as ps
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import json

# from mpl_toolkits.basemap import Basemap

from pandas.io.json import json_normalize
from wordcloud import WordCloud
import squarify
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re 
import gensim 
from gensim import corpora

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.style.use('bmh')

ModuleNotFoundError: No module named 'squarify'

## LOAD DATA FOR ALL BUSINESSES

In [ ]:
businesses = []
with open('datasets/yelp_academic_dataset_business.json', encoding='utf8') as f:
    for line in f:
        businesses.append(json.loads(line))
        
df = json_normalize(businesses)
df.columns = df.columns.map(lambda x: x.split(".")[-1])

df

### TOP WORDS (BY BUSINESS) IN YELP DATASET

In [ ]:
business = df

cloud = WordCloud(width=1440, height= 1080,max_words= 200).generate(' '.join(business['name'].astype(str)))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.axis('off');

### TOP CITIES FOR ALL BUSINESS IN YELP

In [ ]:
print('Number of city listed',business['city'].nunique())
f,ax = plt.subplots(1,2, figsize=(14,8))
ax1,ax2, = ax.flatten()
cnt = business['city'].value_counts()[:20].to_frame()

sns.barplot(cnt['city'], cnt.index, palette = 'gist_rainbow', ax =ax1)
ax1.set_xlabel('')
ax1.set_title('Top city business listed in Yelp')

cnt = business['state'].value_counts()[:20].to_frame()

sns.barplot(cnt['state'], cnt.index, palette = 'coolwarm', ax =ax2)
ax2.set_xlabel('')
ax2.set_title('Top state business listed in Yelp');

### TOP CITIES FOR RESTAURANTS IN YELP

In [ ]:
all_restaurants = df[df['categories'].str.contains("Restaurant") == True]

print('Number of city listed',all_restaurants['city'].nunique())
f,ax = plt.subplots(1,2, figsize=(14,8))
ax1,ax2, = ax.flatten()
cnt = all_restaurants['city'].value_counts()[:20].to_frame()

sns.barplot(cnt['city'], cnt.index, palette = 'gist_rainbow', ax =ax1)
ax1.set_xlabel('')
ax1.set_title('Top city restaurants listed in Yelp')

cnt = all_restaurants['state'].value_counts()[:20].to_frame()

sns.barplot(cnt['state'], cnt.index, palette = 'coolwarm', ax =ax2)
ax2.set_xlabel('')
ax2.set_title('Top state restaurants listed in Yelp');

In [ ]:
restaurants = df[df['categories'].str.contains("Restaurant") == True]
toronto_restaurants = restaurants[restaurants['city'] == 'Toronto']

t_italian = toronto_restaurants[toronto_restaurants['categories'].str.contains("Italian") == True]
t_pizza = toronto_restaurants[toronto_restaurants['categories'].str.contains("Pizza") == True]
t_chinese = toronto_restaurants[toronto_restaurants['categories'].str.contains("Chinese") == True]
t_indian = toronto_restaurants[toronto_restaurants['categories'].str.contains("Indian") == True]
t_steakhouses = toronto_restaurants[toronto_restaurants['categories'].str.contains("Steakhouses") == True]
t_japanese = toronto_restaurants[toronto_restaurants['categories'].str.contains("Japanese") == True]
t_mexican = toronto_restaurants[toronto_restaurants['categories'].str.contains("Mexican") == True]
t_american_new = toronto_restaurants[toronto_restaurants['categories'].str.contains("American \(New\)") == True]
t_greek = toronto_restaurants[toronto_restaurants['categories'].str.contains("Greek") == True]
t_thai = toronto_restaurants[toronto_restaurants['categories'].str.contains("Thai") == True]
t_fast_food = toronto_restaurants[toronto_restaurants['categories'].str.contains("Fast Food") == True]
t_american_trad = toronto_restaurants[toronto_restaurants['categories'].str.contains("American \(Traditional\)") == True]
t_bakeries = toronto_restaurants[toronto_restaurants['categories'].str.contains("Bakeries") == True]
t_canadian_new = toronto_restaurants[toronto_restaurants['categories'].str.contains("Canadian \(New\)") == True]
t_sandwiches = toronto_restaurants[toronto_restaurants['categories'].str.contains("Sandiwiches") == True]
t_asian_fusion = toronto_restaurants[toronto_restaurants['categories'].str.contains("Asian Fusion") == True]

In [ ]:
vegas_restaurants = restaurants[restaurants['city'] == 'Las Vegas']

v_italian = vegas_restaurants[vegas_restaurants['categories'].str.contains("Italian") == True]
v_pizza = vegas_restaurants[vegas_restaurants['categories'].str.contains("Pizza") == True]
v_chinese = vegas_restaurants[vegas_restaurants['categories'].str.contains("Chinese") == True]
v_indian = vegas_restaurants[vegas_restaurants['categories'].str.contains("Indian") == True]
v_steakhouses = vegas_restaurants[vegas_restaurants['categories'].str.contains("Steakhouses") == True]
v_japanese = vegas_restaurants[vegas_restaurants['categories'].str.contains("Japanese") == True]
v_mexican = vegas_restaurants[vegas_restaurants['categories'].str.contains("Mexican") == True]
v_american_new = vegas_restaurants[vegas_restaurants['categories'].str.contains("American \(New\)") == True]
v_greek = vegas_restaurants[vegas_restaurants['categories'].str.contains("Greek") == True]
v_thai = vegas_restaurants[vegas_restaurants['categories'].str.contains("Thai") == True]
v_fast_food = vegas_restaurants[vegas_restaurants['categories'].str.contains("Fast Food") == True]
v_american_trad = vegas_restaurants[vegas_restaurants['categories'].str.contains("American \(Traditional\)") == True]
v_bakeries = vegas_restaurants[vegas_restaurants['categories'].str.contains("Bakeries") == True]
v_canadian_new = vegas_restaurants[vegas_restaurants['categories'].str.contains("Canadian \(New\)") == True]
v_sandwiches = vegas_restaurants[vegas_restaurants['categories'].str.contains("Sandiwiches") == True]
v_asian_fusion = vegas_restaurants[vegas_restaurants['categories'].str.contains("Asian Fusion") == True]

In [ ]:
t_italian['cuisine'] = str("italian")
t_pizza['cuisine'] = str("pizza")
t_chinese['cuisine'] = str("chinese")
t_indian['cuisine'] = str("indian")
t_steakhouses['cuisine'] = str("steakhouses")
t_japanese['cuisine'] = str("japanese")
t_mexican['cuisine'] = str("mexican")
t_american_new['cuisine'] = str("american")
t_greek['cuisine'] = str("greek")
t_thai['cuisine'] = str("thai")
t_fast_food['cuisine'] = str("fast_food")
t_american_trad['cuisine'] = str("american")
t_bakeries['cuisine'] = str("bakeries")
t_canadian_new['cuisine'] = str("canadian")
t_sandwiches['cuisine'] = str("sandwiches")
t_asian_fusion['cuisine'] = str("asian_fusion")

t_restaurants = t_italian.append([t_pizza, t_chinese, t_indian, t_steakhouses, t_japanese, t_mexican, 
                                  t_american_new, t_greek, t_thai, t_fast_food, t_american_trad, 
                                  t_bakeries, t_canadian_new, t_sandwiches, t_asian_fusion])

In [ ]:
v_italian['cuisine'] = str("italian")
v_pizza['cuisine'] = str("pizza")
v_chinese['cuisine'] = str("chinese")
v_indian['cuisine'] = str("indian")
v_steakhouses['cuisine'] = str("steakhouses")
v_japanese['cuisine'] = str("japanese")
v_mexican['cuisine'] = str("mexican")
v_american_new['cuisine'] = str("american")
v_greek['cuisine'] = str("greek")
v_thai['cuisine'] = str("thai")
v_fast_food['cuisine'] = str("fast_food")
v_american_trad['cuisine'] = str("american")
v_bakeries['cuisine'] = str("bakeries")
v_canadian_new['cuisine'] = str("canadian")
v_sandwiches['cuisine'] = str("sandwiches")
v_asian_fusion['cuisine'] = str("asian_fusion")

v_restaurants = v_italian.append([v_pizza, v_chinese, v_indian, v_steakhouses, v_japanese, 
                                  v_mexican, v_american_new, v_greek, v_thai, v_fast_food,
                                  v_american_trad, v_bakeries, v_canadian_new, v_sandwiches, 
                                  v_asian_fusion])

### COMPARE CUISINE RANK BETWEEN TORONTO & LAS VEGAS RESTAURANTS

In [ ]:
f,ax = plt.subplots(1,2, figsize=(14,8))
ax1,ax2,  = ax.flatten()
cnt = t_restaurants['cuisine'].value_counts()[:20].to_frame()

sns.barplot(cnt['cuisine'], cnt.index, palette = 'gist_rainbow', ax =ax1)
ax1.set_xlabel('')
ax1.set_title('Cuisine Ranked for Toronto')


cnt = v_restaurants['cuisine'].value_counts()[:20].to_frame()

sns.barplot(cnt['cuisine'], cnt.index, palette = 'gist_rainbow', ax =ax2)
ax2.set_xlabel('')
ax2.set_title('Cuisine Ranked for Las Vegas')


##### Top cuisine in Toronto is Canadian, at 659 restaurants, while in Las Vegas, Fast Food type restaurant is top with 930 businesses. However, if we combine both American (Trad & New), we'll have a total of 1259 businesses.

In [ ]:
def basic_details(df):
    print('Row:{}, columns:{}'.format(df.shape[0],df.shape[1]))
    k = pd.DataFrame()
    k['number of Unique value'] = df.nunique()
    k['Number of missing value'] = df.isnull().sum()
    k['Data type'] = df.dtypes
    return k

### ALL RESTAURANTS

In [ ]:
basic_details(all_restaurants)

### LAS VEGAS RESTAURANTS

In [ ]:
basic_details(vegas_restaurants)

### TORONTO RESTATURANTS

In [ ]:
basic_details(toronto_restaurants)

### Distribution of rating (ALL B)
Let look at distribution of rating given by the user for different business.

In [ ]:
plt.figure(figsize=(12,4))
ax = sns.countplot(all_restaurants['stars'])
plt.title('Distribution of rating for All Restaurants');

most gives ratings between 3.5 to 4.0

In [ ]:
plt.figure(figsize=(12,4))
ax = sns.countplot(vegas_restaurants['stars'])
plt.title('Distribution of rating for Vegas Restaurants');

In [ ]:
plt.figure(figsize=(12,4))
ax = sns.countplot(toronto_restaurants['stars'])
plt.title('Distribution of rating for Toronto Restaurants');

### REVIEW COUNT FOR VEGAS RESTAURANTS

In [ ]:
print('Median review count',vegas_restaurants['review_count'].median())
plt.figure(figsize = (14,10))
sns.barplot(vegas_restaurants[vegas_restaurants['review_count'] >3000]['review_count'],vegas_restaurants[vegas_restaurants['review_count'] >3000]['name'],
           palette = 'summer')
plt.xlabel('')
plt.title('Top review count of Vegas Restaurants');

### REVIEW COUNT FOR TORONTO RESTAURANTS

In [ ]:
print('Median review count',toronto_restaurants['review_count'].median())
plt.figure(figsize = (14,10))
sns.barplot(toronto_restaurants[toronto_restaurants['review_count'] >500]['review_count'],
            toronto_restaurants[toronto_restaurants['review_count'] >500]['name'],
           palette = 'summer')
plt.xlabel('')
plt.title('Top review count of Toronto Restaurants');

### REVIEW COUNT FOR AMERICAN CUISINE RESTAURANTS IN VEGAS

In [ ]:
len(amer_rest)

In [ ]:
amer_rest = v_american_new.append(v_american_trad)

amer_rest = amer_rest[~amer_rest.index.duplicated(keep='first')]

print('Median review count',amer_rest['review_count'].median())
plt.figure(figsize = (14,10))
sns.barplot(amer_rest[amer_rest['review_count'] >2500]['review_count'],amer_rest[amer_rest['review_count'] >2500]['name'],
           palette = 'summer')
plt.xlabel('')
plt.title('Top review count of American Cuisine Restaurants in Vegas');

In [ ]:
a = ps.sqldf('''select name from amer_rest order by review_count desc limit 10''', locals())
a.head()

### REVIEW COUNT FOR CANADIAN CUISINE RESTAURANTS IN TORONTO

In [ ]:
can_rest = toronto_restaurants[toronto_restaurants['categories'].str.contains("Canadian \(New\)") == True]


print('Median review count',can_rest['review_count'].median())
plt.figure(figsize = (14,10))
sns.barplot(can_rest[can_rest['review_count'] >300]['review_count'],can_rest[can_rest['review_count'] >300]['name'],
           palette = 'summer')
plt.xlabel('')
plt.title('Top review count of Canadian Cuisine Restaurants');

### REVIEW COUNT BY RATINGS

In [ ]:
plt.figure(figsize=(12,4))
ax = sns.countplot(amer_rest['stars'])
plt.title('Distribution of rating for American Cusine in Las Vegas');

In [ ]:
q = """SELECT count(distinct business_id) as no_of_restaurants, stars FROM 
        (select business_id,
        case 
            when stars < 3.5 then 'low'
            when stars = 3.5 then 'med'
            else 'high'
        end as stars
        from amer_rest)
        group by stars """

a = ps.sqldf(q, locals())

a.head()

In [ ]:
plt.figure(figsize=(12,4))
ax = sns.countplot(can_rest['stars'])
plt.title('Distribution of rating for Canadian Cusine in Toronto');

In [ ]:
q1 = """SELECT count(distinct business_id) as no_of_restaurants, stars FROM 
        (select business_id,
        case 
            when stars < 3.5 then 'low'
            when stars = 3.5 then 'med'
            else 'high'
        end as stars
        from can_rest)
        group by stars """

a = ps.sqldf(q1, locals())

a.head()

## CONCLUSION

#### Based on the data exploration done here, it is found that we could get more restaurant datasets if we specify on American Cuisine in Las Vegas.

#### Furthermore, the top restaurant with most reviews for American Cuisine in Vegas is nearly 10 times more than the top restaurant with most reviews for Canadian Cuisine in Toronto.

#### Restaurants with most reviews within the 'American Cuisine in Vegas' dataset is Gordon Ramsay BurGR.

In [ ]:
vegas_restaurants.to_csv("vegas_rest.csv", sep='\t', encoding='utf-8')
amer_rest.to_csv("american_restaurants.csv", sep='\t', encoding='utf-8')